In [1]:
from report import *

from trial_data import extract

reports_dir = '/home/katie/Downloads/galette/slurm-1273660/'
data = extract(reports_dir, reports_dir)[1]

Searching for trials in /home/katie/Downloads/galette/slurm-1273660/.
Found 1680 trials.
Checking trials.
	Unsuccessful trial 557 --- {'benchmark': 'luindex', 'tool': 'mirror-taint', 'status': 'RUN_FAILURE'}
	Unsuccessful trial 598 --- {'status': 'MISSING'}
	Unsuccessful trial 900 --- {'benchmark': 'tradesoap', 'tool': 'mirror-taint', 'status': 'RUN_FAILURE'}
	Unsuccessful trial 1666 --- {'benchmark': 'h2o', 'tool': 'phosphor', 'status': 'RUN_FAILURE'}
	Unsuccessful trial 313 --- {'benchmark': 'xalan', 'tool': 'mirror-taint', 'status': 'RUN_FAILURE'}
	Unsuccessful trial 227 --- {'benchmark': 'tradebeans', 'tool': 'mirror-taint', 'status': 'RUN_FAILURE'}
	Unsuccessful trial 664 --- {'benchmark': 'pmd', 'tool': 'phosphor', 'status': 'RUN_FAILURE'}
	Unsuccessful trial 805 --- {'benchmark': 'h2o', 'tool': 'mirror-taint', 'status': 'RUN_FAILURE'}
	Unsuccessful trial 305 --- {'benchmark': 'luindex', 'tool': 'mirror-taint', 'status': 'RUN_FAILURE'}
	Unsuccessful trial 1522 --- {'status': 'MIS

In [2]:
data[['tool', 'benchmark']].value_counts()

tool          benchmark 
galette       avrora        100
              batik         100
none          eclipse       100
              fop           100
              graphchi      100
              h2            100
              h2o           100
              jme           100
              jython        100
              luindex       100
              lusearch      100
              pmd           100
              spring        100
              sunflow       100
              tomcat        100
              tradebeans    100
              tradesoap     100
              xalan         100
              zxing         100
phosphor      avrora        100
              biojava       100
              fop           100
              graphchi      100
              h2            100
              jme           100
              luindex       100
              lusearch      100
              spring        100
              sunflow       100
none          biojava       100
              b

In [3]:
memory = create_performance_table(data, 'rss')
s_memory = style_performance_table(memory, 'Peak Memory Usage.')
s_memory

In [4]:
print(s_memory.to_latex(multicol_align='c', hrules=True, multirow_align='t', convert_css=True))

\begin{table}
\caption{Peak Memory Usage.}
\begin{tabular}{lrrrrrrrrrrrr}
\toprule
 & \multicolumn{3}{c}{Base} & \multicolumn{3}{c}{Galette} & \multicolumn{3}{c}{MirrorTaint} & \multicolumn{3}{c}{Phosphor} \\
 & MED & LCL & UCL & OV% & LCL & UCL & OV% & LCL & UCL & OV% & LCL & UCL \\
\midrule
avrora & 126,888 & 118,766 & 132,078 & 70.93 & 63.50 & 83.33 & \color{green} 1,672.04 & \color{green} 1,582.68 & \color{green} 1,797.78 & \color{green} 156.12 & \color{green} 145.41 & \color{green} 174.44 \\
batik & 226,222 & 220,376 & 229,780 & 148.23 & 140.18 & 155.98 & \color{green} 924.69 & \color{green} 896.37 & \color{green} 953.57 & --- & --- & --- \\
biojava & 171,582 & 170,632 & 172,595 & 234.19 & 226.56 & 242.17 & \color{green} 400.78 & \color{green} 394.04 & \color{green} 406.27 & \color{red} 130.10 & \color{red} 125.60 & \color{red} 132.60 \\
eclipse & 295,456 & 283,124 & 311,140 & 98.62 & 83.46 & 125.08 & --- & --- & --- & --- & --- & --- \\
fop & 144,340 & 143,276 & 145,324 & 110.41 

In [5]:
time = create_performance_table(data, 'elapsed_time')
s_time = style_performance_table(time, 'Execution Time')
s_time

In [6]:
print(s_time.to_latex(multicol_align='c', hrules=True, multirow_align='t', convert_css=True))

\begin{table}
\caption{Execution Time}
\begin{tabular}{lrrrrrrrrrrrr}
\toprule
 & \multicolumn{3}{c}{Base} & \multicolumn{3}{c}{Galette} & \multicolumn{3}{c}{MirrorTaint} & \multicolumn{3}{c}{Phosphor} \\
 & MED & LCL & UCL & OV% & LCL & UCL & OV% & LCL & UCL & OV% & LCL & UCL \\
\midrule
avrora & 2,360 & 2,284 & 2,414 & 90.40 & 83.56 & 98.34 & \color{green} 66,252.96 & \color{green} 62,858.12 & \color{green} 68,991.94 & \color{green} 159.02 & \color{green} 146.98 & \color{green} 176.07 \\
batik & 272 & 255 & 290 & 1,129.91 & 1,032.88 & 1,221.26 & \color{green} 290,886.79 & \color{green} 271,591.48 & \color{green} 310,119.50 & --- & --- & --- \\
biojava & 134 & 132 & 136 & 2,927.99 & 2,782.44 & 3,030.83 & \color{green} 264,256.72 & \color{green} 242,195.44 & \color{green} 278,104.07 & \color{red} 1,531.72 & \color{red} 1,469.16 & \color{red} 1,622.40 \\
eclipse & 628 & 410 & 5,334 & 168.07 & -69.24 & 332.75 & --- & --- & --- & --- & --- & --- \\
fop & 110 & 102 & 121 & 301.37 & 257.39 